In [7]:
import os
import pandas as pd
from PIL import Image
from sklearn.decomposition import PCA
import numpy as np

In [4]:
# Define paths and parameters
data_dir = f"data\extracted_images"  # Set this to your data directory
output_csv = 'output.csv'
image_size = (45, 45)
n_components = 10  # Set to the number of PCA components you want

<>:2: SyntaxWarning: invalid escape sequence '\e'
<>:2: SyntaxWarning: invalid escape sequence '\e'
C:\Users\anhna\AppData\Local\Temp\ipykernel_19124\1237000731.py:2: SyntaxWarning: invalid escape sequence '\e'
  data_dir = f"data\extracted_images"  # Set this to your data directory


In [10]:
# Initialize PCA and DataFrame
pca = PCA(n_components=n_components)
images = []
metadata = []

In [11]:
# Process each folder (label)
for label in os.listdir(data_dir):
    label_path = os.path.join(data_dir, label)
    if not os.path.isdir(label_path):
        continue
    
    # Process each image in the folder
    for filename in os.listdir(label_path):
        file_path = os.path.join(label_path, filename)
        
        # Open and resize the image, then flatten it
        with Image.open(file_path) as img:
            img = img.resize(image_size).convert('L')  # Resize and convert to grayscale
            img_array = np.array(img).flatten()
            images.append(img_array)  # Collect flattened images for PCA
            
            # Collect metadata for each image
            file_size_kb = os.path.getsize(file_path) / 1024  # File size in KB
            metadata.append([label, filename, file_size_kb])

# Convert images to numpy array
images_array = np.array(images)

# Apply PCA to all images
pca = PCA(n_components=n_components)
pca_features = pca.fit_transform(images_array)

# Combine metadata and PCA features into a single DataFrame
columns = ['Label', 'File Name', 'File Size (KB)'] + [f'PC{i+1}' for i in range(n_components)]
data = [meta + list(features) for meta, features in zip(metadata, pca_features)]
df = pd.DataFrame(data, columns=columns)

# Export to CSV
df.to_csv(output_csv, index=False)
print(f"CSV file saved to {output_csv}")

CSV file saved to output.csv


In [ ]:
# Create DataFrame with column names
columns = ['Label', 'File Name', 'File Size (KB)'] + [f'PC{i+1}' for i in range(n_components)]
df = pd.DataFrame(data, columns=columns)

In [ ]:
# Export to CSV
df.to_csv(output_csv, index=False)
print(f"CSV file saved to {output_csv}")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Load the CSV file
df = pd.read_csv('output_csv.csv')

# Separate features and labels
X = df[[f'PC{i+1}' for i in range(10)]]  # Adjust if using a different number of components
y = df['Label']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Initialize and train AdaBoost classifier
adaboost = AdaBoostClassifier(n_estimators=50, random_state=42)
adaboost.fit(X_train, y_train)

# Make predictions on the test set
y_pred = adaboost.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)

